In [19]:
import streamlit as st
import pandas as pd
from transformers import AutoTokenizer, AutoModel, BertForMaskedLM, BertTokenizer
import tokenizers
from typing import *
import torch
import time


In [5]:
q_a_pairs = list()
with open("../data/def_question.tsv", "r") as f:
    for x in f.readlines():
        row = x.strip().split("\t")
        for i in range(1, len(row)):
            q_a_pairs.append((row[0], row[i]))
q_a_df = pd.DataFrame(q_a_pairs)

In [6]:
q_a_df = pd.DataFrame(q_a_pairs)

In [7]:
q_a_pairs

[('Jak nazywa się pierwsza litera alfabetu greckiego?', 'alfa'),
 ('Jak nazywa się dowolny odcinek łączący dwa punkty okręgu?', 'cięciwa'),
 ('Jak nazywał się sztywny kapelusz męski o zaokrąglonej główce i wąskim lekko uniesionym rondelku, modny w drugiej połowie XIX wieku?',
  'melonik'),
 ('Jak nazywa się trucizna przyrządzona z korzenia szaleju?', 'cykuta'),
 ('Jak nazywa się przyrząd do palenia i górny kieł dzika?', 'fajka'),
 ('Jak nazywa się tradycyjna żydowska czapka?', 'jarmułka'),
 ('Jak nazywa się tradycyjna żydowska czapka?', 'kipa'),
 ('Jak nazywa się tradycyjna żydowska czapka?', 'sztrajmł'),
 ('Jak nazywa się tradycyjna żydowska czapka?', 'sztrajmel'),
 ('Jak nazywa się tradycyjna żydowska czapka?', 'sztrejmł'),
 ('Jak nazywał się przodownik chóru w starożytnym teatrze greckim?',
  'koryfeusz'),
 ('Jak nazywa się jednostka w lotnictwie licząca 2–3 eskadry?', 'dywizjon'),
 ('Jak nazywa się jednostka w lotnictwie licząca 2–3 eskadry?',
  'pułk lotniczy'),
 ('Jak nazywa się 

In [9]:
def predict(
    model: AutoModel, 
    tokenizer: AutoTokenizer,
    sentence: List[str],
) -> torch.tensor:
    return model(
        **tokenizer.batch_encode_plus(sentence,
        padding='longest',
        add_special_tokens=True,
        return_tensors='pt'
        )
    )['pooler_output']
    #.values().mapping['pooler_output']

def vectorize_and_save(
    dataset: List[Tuple[str, str]],
    model: AutoModel, 
    tokenizer: AutoTokenizer, 
    #path_and_filename: str,
    #from_index: int = 0, 
) -> None:
    BATCH_SIZE = 300
    for i in range(0, len(dataset), BATCH_SIZE):
        st.text(f'Batch {i} -> {i + BATCH_SIZE - 1}')
        encodings = predict(
            model=model,
            tokenizer=tokenizer,
            sentence=[q+a for q, a in dataset[i:i + BATCH_SIZE]]
        )
        #torch.save(encodings, f'{path_and_filename}_{i}.pt')
        #del encodings
    return encodings

In [12]:
def load_herbert() -> Tuple[AutoTokenizer, AutoModel]:
    tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")
    model = AutoModel.from_pretrained("allegro/herbert-large-cased")
    return tokenizer, model

In [14]:
herbert_tokenizer, herbert_model = load_herbert()

encodings = vectorize_and_save(
             dataset=q_a_pairs,
             model=herbert_model,
             tokenizer=herbert_tokenizer,
         )

Some weights of the model checkpoint at allegro/herbert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-11-21 00:29:41.574 
  command:

    streamlit run /opt/anaconda3/envs/conv-agent/lib/python3.10/site

In [20]:
"[CLS] " + q_a_pairs[0][0] + " [SEP] " + q_a_pairs[0][1] + " [SEP]"

'[CLS] Jak nazywa się pierwsza litera alfabetu greckiego? [SEP] alfa [SEP]'

In [21]:
def join_encoding(
    to_index: int,
    filename: str,
) -> torch.Tensor:
    BATCH_SIZE = 100
    encoded_data = torch.load(f'../data_encoded/{filename}_0.pt')
    for i in range(100, to_index, BATCH_SIZE):
        encoded = torch.load(f'../data_encoded/{filename}_{i}.pt')
        encoded_data = torch.cat([encoded_data, encoded])
    return encoded_data

encoded_qa_data = join_encoding(
        to_index=4000,
        filename="q_a/encoded" 
    )

In [37]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30, random_state=0).fit(encoded_qa_data.detach().numpy().astype('double'))

In [38]:
q = encoded_qa_data[0].detach().numpy()
kmeans.predict(X = [q])

array([21], dtype=int32)